### Qarpo 
This notebook provides a recipe to use the qarpo library to build a user interface to submit jobs to job scheduler and display output interface to display accomplished jobs' outputs

To start using the qarpo library features, run the following cell to import qarpo

In [ ]:
import qarpo

qarpo is a tool to submit jobs to nodes in a cluster from the jupyter notebook, the first step is to prepare the job script to be submitted to the job scheduler.

The following cell writes the instructions that will run on the target node into app/example_job.sh

In [ ]:
%%writefile app/example_job.sh

# The default path for the job is your home directory, so we change directory to where the files are.
cd $PBS_O_WORKDIR
mkdir -p $1

OUTPUT_FILE=$1
DEVICE=$2
FP_MODEL=$3


if [ "$2" = "HETERO:FPGA,CPU" ]; then
    # Environment variables and compilation for edge compute nodes with FPGAs
    export LD_LIBRARY_PATH=${LD_LIBRARY_PATH}:/opt/altera/aocl-pro-rte/aclrte-linux64/
    source /opt/fpga_support_files/setup_env.sh
    aocl program acl0 /opt/intel/computer_vision_sdk/bitstreams/a10_vision_design_bitstreams/5-0_PL1_FP11_MobileNet_Clamp.aocx
fi
# Running the object detection code
SAMPLEPATH=$PBS_O_WORKDIR
python3 app/classification_sample.py  -m app/model/${FP_MODEL}/crnn.xml  \
                                           -i app/board4.jpg \
                                           -o $OUTPUT_FILE \
                                           -d $DEVICE
                                           

The following cell is building the output interface using the Interface class from qarpo.

The Intefrace class receive an input dictionary with the configuration of the user interface.

The first key in the input dictionary is "job", the corresponding value is a dictionary of required information for the interface.

    {
        "job":
            {
                "output_type": "type of the job output (.txt, .png, .mp4)",
                "results_path": <path to the directory where jobs will write its ouutput>
            }
     }

In [ ]:
job_interface = qarpo.Interface( {
                        "job": # Define how to launch the job and interpret results
                                {
                                    "output_type": ".txt",         # The type of input (text/video)
                                    "results_path": "app/results/" # Path to job result files
                                   
                                }
                        } )

After creating the interface configuration, display the UI using the displayUI function. 
In this basic recipe, the displayUI function will show an empty box, this is the placeholder that will hold the output interface when you start submitting jobs to the queue.

In [ ]:
job_interface.displayUI()

To start submitting jobs to the queue, use the submitJob() function, this function is part of qarpo library.
It takes a string as an input, in our case, this string is the qsub command to submit jobs to target nodes in the cluster

In [ ]:
job_interface.submitJob("qsub app/example_job.sh -l nodes=1:idc001skl:i5-6500te -F 'app/results/ GPU FP32'")

In [ ]:
job_interface.submitJob("qsub app/example_job.sh -l nodes=1:idc001skl:i5-6500te -F 'app/results/ CPU FP32'")